# Aula 1: Arquitetura RAG na Prática com Gemini

## O que vamos aprender:
- Como o RAG resolve problemas reais dos LLMs
- Diferenças práticas entre RAG e prompting tradicional
- Primeiros passos com LangChain e Google Gemini
- Como carregar e processar documentos PDF

### Por que RAG é essencial no mercado?
1. **Dados sempre atualizados** - Sem retreinar modelos
2. **Respostas verificáveis** - Com fontes de informação
3. **Custo otimizado** - Evita fine-tuning caro
4. **Aplicações reais**: Chatbots de suporte, análise de documentos, assistentes internos

## 0. Configuração do Ambiente

Primeiro, vamos instalar as bibliotecas necessárias. Usaremos:
- `langchain` e `langchain-google-genai` para interagir com o Gemini.
- `pypdf` para ler o conteúdo do nosso arquivo PDF.


In [1]:
!pip install langchain langchain-google-genai==2.0.5 google-generativeai pypdf
!pip install langchain-community

INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.2/457.2 kB 28.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 1.2.5
    Uninstalling langchain-core-1.2.5:
      Successfully uninstalled langchain-core-1.2.5
  Attempting uninstall: langchain-text-s

- Onde obter chave google: https://aistudio.google.com/apikey

In [2]:
import os

os.environ['GOOGLE_API_KEY'] = '...'

## 1. Prompting Tradicional vs RAG - Comparação Prática

Vamos ver na prática a diferença entre usar apenas um LLM (prompting tradicional) e usar RAG.

In [16]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature = 0)

In [17]:
# EXEMPLO 1: Prompting Tradicional (sem RAG)

pergunta = "Qual é a política de home office da nossa empresa?"

prompt_tradicional = ChatPromptTemplate.from_template(
    "Responda a seguinte pergunta: {pergunta}"
)

In [18]:
chain_tradicional = prompt_tradicional | llm

resposta_tradicional = chain_tradicional.invoke({"pergunta": pergunta})

In [19]:
print(resposta_tradicional.content)

Como sou uma inteligência artificial e não tenho acesso às políticas internas específicas da sua empresa, não consigo te dar a resposta exata.

No entanto, posso te explicar os tipos comuns de políticas de home office que as empresas adotam hoje em dia, para que você tenha uma ideia do que procurar:

1.  **Totalmente Remoto (Full Remote):** Todos os colaboradores trabalham de casa permanentemente, independentemente da função. A empresa pode ter um escritório físico, mas ele não é o local principal de trabalho.
2.  **Híbrido (Hybrid):** Esta é a modalidade mais comum atualmente e pode ter várias variações:
    *   **Dias Fixos:** Os colaboradores têm dias específicos da semana para ir ao escritório (ex: terças e quintas) e os demais dias trabalham de casa.
    *   **Dias Flexíveis:** Os colaboradores precisam cumprir um número mínimo de dias no escritório por mês ou semana, mas podem escolher quais dias.
    *   **Baseado em Equipe:** Cada equipe ou departamento define sua própria polít

-----
## OBS: O que é esse | ?

O operador `|` em Python, nesse contexto com **LangChain**, é uma **forma elegante de compor etapas de uma cadeia (chain)** de execução, como se fosse um **"pipeline"**.

### ✅ Significado prático no LangChain:

```python
chain = prompt | llm
```

Esse código cria uma **cadeia (chain)** onde:

* O `prompt` é executado primeiro,
* E o resultado (texto formatado com as variáveis) é enviado diretamente para o `llm` (modelo de linguagem, como o Gemini),
* Retornando a **resposta gerada**.

-------

## Carregando nosso Documento
Agora, em vez de simular o documento, vamos carregar o `politica_home_office.pdf` que está na mesma pasta.

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("politica_home_office.pdf")

documento = loader.load()

In [ ]:
documento

[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-07-07T09:42:25-03:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-07-07T09:42:25-03:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'politica_home_office.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content="Política de Trabalho Remoto e Híbrido\nVersão 2.1 - Atualizada em Janeiro 2024\n1. Objetivo\nEsta política estabelece as diretrizes para o trabalho remoto e híbrido na Empresa XYZ, visando\npromover a flexibilidade, o bem-estar dos funcionários e a manutenção da produtividade e\ncolaboração.\n2. Elegibilidade\nTodos os funcionários em tempo integral, que completaram o período de experiência de 90 dias e\ncujas funções são compatíveis com o trabalho remoto, são elegíveis para aderir ao modelo de trabalho\nhíbrido. A aprovação final está sujeita ao acordo com o gestor direto.\n3. Modalidade

In [ ]:
contexto_empresa = documento[0].page_content

In [ ]:
contexto_empresa

"Política de Trabalho Remoto e Híbrido\nVersão 2.1 - Atualizada em Janeiro 2024\n1. Objetivo\nEsta política estabelece as diretrizes para o trabalho remoto e híbrido na Empresa XYZ, visando\npromover a flexibilidade, o bem-estar dos funcionários e a manutenção da produtividade e\ncolaboração.\n2. Elegibilidade\nTodos os funcionários em tempo integral, que completaram o período de experiência de 90 dias e\ncujas funções são compatíveis com o trabalho remoto, são elegíveis para aderir ao modelo de trabalho\nhíbrido. A aprovação final está sujeita ao acordo com o gestor direto.\n3. Modalidade e Horário\n3.1. Modelo Híbrido: A modalidade padrão é híbrida, compreendendo 3 (três) dias de trabalho\nremoto (home office) e 2 (dois) dias de trabalho presencial no escritório, por semana.\n3.2. Dias Presenciais: Os dias de trabalho presencial serão definidos em comum acordo entre a\nequipe e o gestor, priorizando as terças-feiras para reuniões de alinhamento geral da equipe.\n3.3. Horário Flexível

In [ ]:
print(contexto_empresa[:500] + "...")

Política de Trabalho Remoto e Híbrido
Versão 2.1 - Atualizada em Janeiro 2024
1. Objetivo
Esta política estabelece as diretrizes para o trabalho remoto e híbrido na Empresa XYZ, visando
promover a flexibilidade, o bem-estar dos funcionários e a manutenção da produtividade e
colaboração.
2. Elegibilidade
Todos os funcionários em tempo integral, que completaram o período de experiência de 90 dias e
cujas funções são compatíveis com o trabalho remoto, são elegíveis para aderir ao modelo de trabalho...


In [ ]:
# Exemplo 2: Com RAG - Usando o contexto do PDF

prompt_rag = ChatPromptTemplate.from_template("""
Use o contexto abaixo para responder a pergunta.
Se não souber a resposta baseado no contexto, diga que não tem a informação.

Contexto: {contexto}
Pergunta: {pergunta}

RespostaResposta:""")

In [ ]:
chain_rag = prompt_rag | llm

resposta_rag = chain_rag.invoke({"contexto": contexto_empresa, "pergunta": pergunta})

In [ ]:
print(resposta_rag.content)

A política da empresa XYZ não é especificamente de "home office", mas sim de trabalho híbrido.  Funcionários elegíveis trabalham 3 dias remotamente (home office) e 2 dias presencialmente no escritório por semana.  A modalidade padrão exige a presença no escritório às terças-feiras para reuniões de equipe.  Funcionários em tempo integral que completaram o período de experiência de 90 dias e cujas funções são compatíveis com o trabalho remoto são elegíveis, mediante aprovação do gestor.


## 📚 Resumo Prático da Aula 1

### O que aprendemos:
1. **RAG vs. Prompting Tradicional**: Vimos na prática como o RAG fornece respostas mais precisas usando dados externos.
2. **LangChain + Gemini**: Configuramos o ambiente para usar os modelos de LLM e Embedding do Google.
3. **Leitura de PDFs**: Aprendemos a carregar o conteúdo de um documento PDF para usar como contexto.
